<a href="https://colab.research.google.com/github/NMashalov/2023_OpenMipt_course/blob/main/lesson8/sem_FE_empty.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd

from sklearn.metrics import mean_squared_error as MSE
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Lasso
from sklearn.ensemble import RandomForestRegressor
import seaborn as sns

sns.set(context='poster')
%matplotlib inline

### Задача 1:

Рассмотрим как можно провести отбор признаков с помощью обучения линейной регрессии и леса с малым количеством деревьев на примере задачи регрессии. Будем использовать датасет <https://www.kaggle.com/datasets/abrambeyer/openintro-possum> и пытаться предсказать возраст оппосумов на основе различных параметров их тела.

Для упрощения задачи избавимся от категориальных признаков

In [2]:
!pip install opendatasets -q

In [3]:
import opendatasets as od

od.download('https://www.kaggle.com/datasets/abrambeyer/openintro-possum')

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username: mashalovne
Your Kaggle Key: ··········


100%|██████████| 2.15k/2.15k [00:00<00:00, 3.67MB/s]

In [6]:
data = pd.read_csv('openintro-possum/possum.csv').dropna()
X = data[data.columns.drop(['sex', 'Pop', 'age'])]
y = data['age']

Разделим выборку на тренировочную и тестовую часть

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

Стандартизируем данные для корректной работы линейной регрессии:

In [19]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train_norm = scaler.fit_transform(X_train)
X_test_norm = scaler.transform(X_test)

Обучим модель линейной регрессии с L1-регуляризацией на всех данных, посмотрим на метрику качества

In [30]:
from sklearn.linear_model import Lasso
lin_model = Lasso(alpha=0.1).fit(X_train,y_train)
print('Коэффиценты модели:', lin_model.coef_)
print('Значение MSE на тренировочной выборке:', MSE(lin_model.predict(X_train_norm), y_train))
print('Значение MSE на тестовой выборке:', MSE(lin_model.predict(X_test_norm), y_test))

Коэффиценты модели: [-0.0215647   0.21001336  0.08407756  0.05269576 -0.         -0.
 -0.15057077  0.08691628  0.16517017  0.          0.15754382]
Значение MSE на тренировочной выборке: 147.7999211446977
Значение MSE на тестовой выборке: 151.61153891046658


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but Lasso was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but Lasso was fitted with feature names
  warnings.warn(


Проделаем аналогичную процедуру с лесом c небольшим количеством деревьев:

In [31]:
from sklearn.ensemble import RandomForestRegressor
forest =  RandomForestRegressor(n_estimators=10).fit(X_train,y_train)
print('Важность признаков:', forest.feature_importances_)
print('Значение MSE на тренировочной выборке:', MSE(forest.predict(X_train), y_train))
print('Значение MSE на тестовой выборке:', MSE(forest.predict(X_test), y_test))

Важность признаков: [0.08730419 0.00711945 0.16815096 0.28876889 0.08033358 0.04137316
 0.07938556 0.08041343 0.08623102 0.00493033 0.07598942]
Значение MSE на тренировочной выборке: 0.5591044776119402
Значение MSE на тестовой выборке: 2.657058823529412


Составим 2 списка наиболее важных признаков - один с признаками, наиболее важными для линейной регрессии, второй с наиболее важными для леса

In [35]:
lin_imp = []
forest_imp = []
for i, column in enumerate(X.columns):
    if abs(lin_model.coef_[i]) > 0.1: # i added model as importance is sign-idependent
        lin_imp.append(i)
    if forest.feature_importances_[i] > 0.1:
        forest_imp.append(column)

Теперь обучим модели только на важных признаках:

In [36]:
X_train_norm_lin_imp = X_train_norm[:, lin_imp]
X_test_norm_lin_imp = X_test_norm[:, lin_imp]

X_train_forest_imp = X_train[forest_imp]
X_test_forest_imp = X_test[forest_imp]

In [37]:
lin_model_2 = Lasso(alpha=0.1).fit(X_train_norm_lin_imp, y_train)
print('Коэффиценты модели:', lin_model_2.coef_)
print('Значение MSE на тренировочной выборке:', MSE(lin_model_2.predict(X_train_norm_lin_imp), y_train))
print('Значение MSE на тестовой выборке:', MSE(lin_model_2.predict(X_test_norm_lin_imp), y_test))

Коэффиценты модели: [ 0.         -0.0123311   0.3047265   0.45966172]
Значение MSE на тренировочной выборке: 3.1845617227795286
Значение MSE на тестовой выборке: 2.9906052916563404


In [41]:
forest_2 = RandomForestRegressor(n_estimators=10).fit(X_train_forest_imp,y_train)
print('Важность признаков:', forest_2.feature_importances_)
print('Значение MSE на тренировочной выборке:', MSE(forest_2.predict(X_train_forest_imp), y_train))
print('Значение MSE на тестовой выборке:', MSE(forest_2.predict(X_test_forest_imp), y_test))

Важность признаков: [0.51681292 0.48318708]
Значение MSE на тренировочной выборке: 0.5580597014925374
Значение MSE на тестовой выборке: 2.219117647058823


**Вывод:**
Отбор признаков позволяет значительно улучшить качество предсказания для линейных моделей. Предсказания случайного леса улучшились незначительно. Сужение выборки и признакового пространства при обучении для каждого элемента ансамбля позволяют случайному леса давать качественные предсказния без необходимости в отборе признаков.